In [3]:
%matplotlib auto

import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from matplotlib.widgets import PolygonSelector
from matplotlib.path import Path

Using matplotlib backend: Qt5Agg


In [18]:
x = np.random.randint(1, 35, 10)
y = np.random.randint(1, 35, 10)  # 随机生成散点
fig, ax = plt.subplots(figsize=(8, 6), dpi=100)  # 创建画布

def onselect(verts):
    print(verts)  # 没有做过多处理，直接打印选择的区域坐标

ax.grid()
ax.plot(x, y, '.')

poly = PolygonSelector(ax, onselect)  # 创建选择器

plt.show()

[(4.727419354838709, 11.800000000000002), (15.851612903225805, 16.53333333333333), (16.383870967741935, 7.466666666666667), (3.662903225806451, 5.2666666666666675), (4.408064516129032, 15.266666666666667), (3.1838709677419352, 9.533333333333335)]
[(4.727419354838709, 11.800000000000002), (15.851612903225805, 16.53333333333333), (16.383870967741935, 7.466666666666667), (3.662903225806451, 5.2666666666666675), (4.408064516129032, 15.266666666666667), (3.1838709677419352, 9.533333333333335)]


In [19]:
# 官方的例子
class SelectFromCollection: # 创建选择器对象
    def __init__(self, ax, collection, alpha_other=0.3):  # 初始化需要坐标轴对象ax和PathCollection对象
        self.canvas = ax.figure.canvas
        self.collection = collection
        self.alpha_other = alpha_other  # 这个参数是设定没有选择到的点的透明度，选择到的点就是0

        self.xys = collection.get_offsets()  # 获取偏移
        self.Npts = len(self.xys)

        # Ensure that we have separate colors for each object
        self.fc = collection.get_facecolors()  # 前景颜色
        if len(self.fc) == 0:
            raise ValueError('Collection must have a facecolor')
        elif len(self.fc) == 1:
            self.fc = np.tile(self.fc, (self.Npts, 1))

        self.poly = PolygonSelector(ax, self.onselect)
        self.ind = []

    def onselect(self, verts):  # 识别选择到的点，重新画图
        path = Path(verts)
        self.ind = np.nonzero(path.contains_points(self.xys))[0]
        self.fc[:, -1] = self.alpha_other
        self.fc[self.ind, -1] = 1
        self.collection.set_facecolors(self.fc)
        self.canvas.draw_idle()

    def disconnect(self):  # 取消选择器连接
        self.poly.disconnect_events()
        self.fc[:, -1] = 1
        self.collection.set_facecolors(self.fc)
        self.canvas.draw_idle()


fig, ax = plt.subplots()
grid_size = 5
grid_x = np.tile(np.arange(grid_size), grid_size)
grid_y = np.repeat(np.arange(grid_size), grid_size)
pts = ax.scatter(grid_x, grid_y)

selector = SelectFromCollection(ax, pts)

plt.show()
